# Preprocessing

In [9]:
import os
import cv2
import numpy as np
from tensorflow.keras.preprocessing import image
from tqdm.auto import tqdm
import tensorflow as tf
from tensorflow.keras.layers import Rescaling
from sklearn.model_selection import train_test_split

c:\Users\fabri\AppData\Local\Programs\Python\Python312\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [10]:
# Define function to remove black borders
def crop_black_borders(image, threshold=10):
    gray = cv2.cvtColor(image, cv2.COLOR_RGB2GRAY)
    _, mask = cv2.threshold(gray, threshold, 255, cv2.THRESH_BINARY)
    coords = cv2.findNonZero(mask)
    if coords is not None:
        x, y, w, h = cv2.boundingRect(coords)
        return image[y:y+h, x:x+w]
    else:
        return image  # fallback if image is entirely black

# Input and output paths
input_dir = "Data/images" 
output_dir = "Data/cleaned_images"
os.makedirs(output_dir, exist_ok=True)

class_folders = [cls for cls in sorted(os.listdir(input_dir)) if os.path.isdir(os.path.join(input_dir, cls))]

# Process each class
for cls in tqdm(class_folders, desc="Processing classes", position=0):
    cls_input_path = os.path.join(input_dir, cls)
    cls_output_path = os.path.join(output_dir, cls)
    os.makedirs(cls_output_path, exist_ok=True)

    image_files = [f for f in os.listdir(cls_input_path) if f.lower().endswith((".jpg", ".jpeg", ".png"))]

    for fname in tqdm(image_files, desc=f"→ {cls:>12}", position=1, leave=True):
        img_path = os.path.join(cls_input_path, fname)
        img_bgr = cv2.imread(img_path)
        if img_bgr is None:
            print(f"Skipped unreadable image: {img_path}")
            continue

        img_rgb = cv2.cvtColor(img_bgr, cv2.COLOR_BGR2RGB)
        cleaned = crop_black_borders(img_rgb)
        cleaned_bgr = cv2.cvtColor(cleaned, cv2.COLOR_RGB2BGR)

        save_path = os.path.join(cls_output_path, fname)
        cv2.imwrite(save_path, cleaned_bgr, [cv2.IMWRITE_JPEG_QUALITY, 95])

Processing classes: 100%|██████████| 12/12 [2:07:08<00:00, 635.70s/it]


In [5]:
# Configuration for train-validation split
data_dir = "C:/Users/fabri/Desktop/uni/MSDS/2024-2025/Semester 2/Advanced Analytics in a Big Data World/Project/Assignment 2/Data/patches"
batch_size = 32
img_height = 224
img_width = 224
val_split = 0.2
seed = 50

# Train-validation split
train_ds = tf.keras.preprocessing.image_dataset_from_directory(
    data_dir,
    validation_split=val_split,
    subset="training",
    seed=seed,
    image_size=(img_height, img_width),
    batch_size=batch_size,
    label_mode="categorical"
)

val_ds = tf.keras.preprocessing.image_dataset_from_directory(
    data_dir,
    validation_split=val_split,
    subset="validation",
    seed=seed,
    image_size=(img_height, img_width),
    batch_size=batch_size,
    label_mode="categorical"
)

Found 2647 files belonging to 12 classes.
Using 2118 files for training.
Found 2647 files belonging to 12 classes.
Using 529 files for validation.


In [6]:
from tensorflow.keras.layers import Rescaling
from tensorflow.keras import layers

# Normalisation and data augmentation
normalisation_layer = Rescaling(1./255)
data_augmentation = tf.keras.Sequential([
    layers.RandomFlip("horizontal"),
    layers.RandomRotation(0.05),
    layers.RandomZoom(0.1),
    layers.RandomContrast(0.1)
])

train_ds = train_ds.map(lambda x, y: (normalisation_layer(data_augmentation(x)), y))
val_ds = val_ds.map(lambda x, y: (normalisation_layer(x), y))

# ResNet50 Model

In [7]:
# Load pretrained ResNet50 model
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.applications.resnet50 import preprocess_input

base_model = ResNet50(
    include_top=False, # Remove the fully-connected layers used for ImageNet classification
    weights="imagenet", # Load pretrained weights from ImageNet
    input_shape=(img_height, img_width, 3),
    pooling="avg" # Global average pooling layer to convert 3D tensor to 1D
)
base_model.trainable = False # Freeze the base model for now

In [ ]:
# Build the model
from tensorflow.keras import Model

num_classes = 12 # 12 neurons, one per country class

inputs = tf.keras.Input(shape=(img_height, img_width, 3)) # Input layer
x = preprocess_input(inputs) # Preprocess input for ResNet50
x = base_model(x, training=False) # Pass the input through the base model
x = layers.Dense(128)(x) # Add a fully connected layer with 128 neurons
x = layers.BatchNormalization()(x) # Add batch normalisation layer
x = layers.Activation("relu")(x) # Add ReLU activation function
x = layers.Dropout(0.3)(x) # Add dropout layer with 30% rate
outputs = layers.Dense(num_classes, activation="softmax")(x) # Add output layer with softmax activation

model = Model(inputs, outputs)

# Compile the model
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.001), # Adam optimiser with learning rate of 0.001 for faster convergence
    loss=tf.keras.losses.CategoricalCrossentropy(label_smoothing=0.1), # Categorical cross-entropy loss with label smoothing of 0.1 to prevent overfitting
    metrics=["accuracy"]
)

# Add callbacks to prevent overfitting, adjust learning rate and save the best model
callbacks = [
    tf.keras.callbacks.EarlyStopping(patience=5, restore_best_weights=True), # Stop training after 5 epochs with no improvement in validation loss
    tf.keras.callbacks.ReduceLROnPlateau(patience=3, factor=0.2), # Reduce learning rate by 20% after 3 epochs with no improvement in validation loss
    tf.keras.callbacks.ModelCheckpoint("best_model_trial_2.keras", save_best_only=True) # Save the best model based on validation loss
]

In [17]:
# Train the model
history = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=20,
    callbacks=callbacks
)

Epoch 1/20
67/67 ━━━━━━━━━━━━━━━━━━━━ 494s 7s/step - accuracy: 0.2455 - loss: 2.4215 - val_accuracy: 0.0851 - val_loss: 431.7726 - learning_rate: 0.0010
Epoch 2/20
67/67 ━━━━━━━━━━━━━━━━━━━━ 464s 7s/step - accuracy: 0.4066 - loss: 1.9133 - val_accuracy: 0.0964 - val_loss: 26.2216 - learning_rate: 0.0010
Epoch 3/20
67/67 ━━━━━━━━━━━━━━━━━━━━ 451s 7s/step - accuracy: 0.4402 - loss: 1.8391 - val_accuracy: 0.0964 - val_loss: 4.9359 - learning_rate: 0.0010
Epoch 4/20
67/67 ━━━━━━━━━━━━━━━━━━━━ 455s 7s/step - accuracy: 0.4983 - loss: 1.6806 - val_accuracy: 0.1172 - val_loss: 2.7445 - learning_rate: 0.0010
Epoch 5/20
67/67 ━━━━━━━━━━━━━━━━━━━━ 490s 7s/step - accuracy: 0.5466 - loss: 1.6167 - val_accuracy: 0.1040 - val_loss: 14.1716 - learning_rate: 0.0010
Epoch 6/20
67/67 ━━━━━━━━━━━━━━━━━━━━ 446s 7s/step - accuracy: 0.5687 - loss: 1.5588 - val_accuracy: 0.0983 - val_loss: 3.9955 - learning_rate: 0.0010
Epoch 7/20
67/67 ━━━━━━━━━━━━━━━━━━━━ 450s 7s/step - accuracy: 0.5939 - loss: 1.4505 - val

In [1]:
# Plot model accuracy and model loss
import matplotlib.pyplot as plt

plt.plot(history.history["accuracy"], label="Training Accuracy")
plt.plot(history.history["val_accuracy"], label="Validation Accuracy")
plt.title("Model Accuracy")
plt.xlabel("Epoch")
plt.ylabel("Accuracy")
plt.legend()
plt.grid()
plt.show()

plt.plot(history.history["loss"], label="Training Loss")
plt.plot(history.history["val_loss"], label="Validation Loss")
plt.title("Model Loss")
plt.xlabel("Epoch")
plt.ylabel("Loss")
plt.legend()
plt.grid()
plt.show()

NameError: name 'history' is not defined

# Fine-tuning

In [15]:
# Unfreeze the base model
base_model.trainable = True

# Recompile with lower learning rate and use Focal Loss
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=1e-5),
    loss=tf.losses.BinaryFocalCrossentropy(gamma=2.0),
    metrics=["accuracy"]
)

# Callbacks
fine_tune_callbacks = [
    tf.keras.callbacks.EarlyStopping(patience=6, restore_best_weights=True),
    tf.keras.callbacks.ReduceLROnPlateau(patience=3, factor=0.2),
    tf.keras.callbacks.ModelCheckpoint("best_model_focal.keras", save_best_only=True),
    tf.keras.callbacks.CSVLogger("training_focal.csv")
]

# Train the model with fine-tuning
fine_tune_epochs = 10
initial_epochs = len(history.epoch)

history_fine = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=initial_epochs + fine_tune_epochs,
    initial_epoch = initial_epochs,
    callbacks=fine_tune_callbacks
)


Epoch 8/17
67/67 ━━━━━━━━━━━━━━━━━━━━ 407s 6s/step - accuracy: 0.3145 - loss: 0.0519 - val_accuracy: 0.0926 - val_loss: 0.1171 - learning_rate: 1.0000e-05
Epoch 9/17
67/67 ━━━━━━━━━━━━━━━━━━━━ 388s 6s/step - accuracy: 0.3542 - loss: 0.0468 - val_accuracy: 0.0907 - val_loss: 0.1907 - learning_rate: 1.0000e-05
Epoch 10/17
67/67 ━━━━━━━━━━━━━━━━━━━━ 388s 6s/step - accuracy: 0.3981 - loss: 0.0425 - val_accuracy: 0.1002 - val_loss: 0.1281 - learning_rate: 1.0000e-05
Epoch 11/17
67/67 ━━━━━━━━━━━━━━━━━━━━ 388s 6s/step - accuracy: 0.4278 - loss: 0.0388 - val_accuracy: 0.0983 - val_loss: 0.0940 - learning_rate: 1.0000e-05
Epoch 12/17
67/67 ━━━━━━━━━━━━━━━━━━━━ 388s 6s/step - accuracy: 0.4637 - loss: 0.0352 - val_accuracy: 0.0832 - val_loss: 0.0814 - learning_rate: 1.0000e-05
Epoch 13/17
67/67 ━━━━━━━━━━━━━━━━━━━━ 387s 6s/step - accuracy: 0.4874 - loss: 0.0322 - val_accuracy: 0.1153 - val_loss: 0.0720 - learning_rate: 1.0000e-05
Epoch 14/17
67/67 ━━━━━━━━━━━━━━━━━━━━ 0s 5s/step - accuracy: 0.49

KeyboardInterrupt: 